In [3493]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from imblearn.over_sampling import RandomOverSampler



In [3494]:
import warnings
warnings.filterwarnings("ignore")


In [3495]:
df_ufil_pred = pd.read_csv("../data/unfilled.csv")

target = pd.read_csv("../data/filtered_df.csv")['ICU']

In [3496]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Display the head of the dataframe with all columns
df_ufil_pred.head()

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
0,1,1,57,0.0,0,160.0,55.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,NaN,1.0,7.0,110.0,60.0,65.0,14.0,37.8,4890.0,4210000.0,11.4,132000.0,99.0,29.0,0.94,338.0,31.0,33.0,1.08,NaN,136.0,3.4,8.40,1.15,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,1,0,0.0,0,0,0.0,NaN
1,0,0,62,0.0,0,159.0,70.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,NaN,1.0,7.0,140.0,90.0,124.0,14.0,38.5,5240.0,4980000.0,14.2,165000.0,104.0,40.0,1.00,260.0,31.0,39.0,0.82,NaN,140.0,3.9,9.40,1.19,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,1,0,0.0,0,0,0.0,NaN
2,1,0,72,0.0,0,175.0,83.0,1,0,0,0,0,0,1,0,0,0,0,0,0.0,0,0,1,1,0.0,7.0,0.2,2.0,80.0,60.0,87.0,NaN,39.0,6000.0,3010000.0,9.4,123000.0,107.0,38.0,0.91,474.0,18.0,18.0,1.75,0.36,139.0,4.0,8.13,1.10,369.0,7.41,30.4,2.6,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,1.0
3,1,0,53,0.0,0,177.0,96.0,1,1,0,0,0,0,0,0,0,0,1,0,0.0,0,1,2,1,0.0,572000.0,12.0,7.0,125.0,85.0,NaN,NaN,40.4,7040.0,4770000.0,14.4,19000.0,230.0,NaN,3.22,NaN,NaN,80.0,1.94,1.02,122.0,3.5,NaN,1.10,407.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,NaN
4,0,1,58,0.0,0,160.0,64.3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,1,1,1,0.0,602100.0,13.5,7.0,107.0,40.0,125.0,16.0,40.0,2810.0,4460.0,14.5,58000.0,115.0,48.0,1.30,NaN,NaN,37.0,1.48,NaN,135.0,3.9,NaN,1.10,455.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,NaN


In [3497]:
df_ufil_pred.shape[1]

65

In [3498]:
empty_value_count = df_ufil_pred.isna().sum().sum()
nb_total_values = df_ufil_pred.shape[0]*df_ufil_pred.shape[1]

In [3499]:
print("count is : " + str(target.count()) + " & sum is "+str(target.sum()))
print("empty_value_% = "+str(empty_value_count / nb_total_values))

count is : 72 & sum is 14
empty_value_% = 0.12756410256410255


In [3500]:
# Number of folds
num_folds = 6

In [3501]:
from sklearn.model_selection import KFold
import random

while True:
    # Initialize the KFold splitter with a random state
    random_state = random.randint(1, 10000)
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)

    trains_df = []
    tests_df = []
    targets_train_df = []
    targets_test_df = []

    # Split the data into 6 folds
    for train_index, test_index in kf.split(df_ufil_pred):
        # Create dataframes for training and testing
        train_df = df_ufil_pred.iloc[train_index]
        test_df = df_ufil_pred.iloc[test_index]

        # Append the training and testing dataframes to lists
        trains_df.append(train_df)
        tests_df.append(test_df)

        # Split the target into training and testing
        target_train_df = target.iloc[train_index]
        target_test_df = target.iloc[test_index]

        # Append the target dataframes to lists
        targets_train_df.append(target_train_df)
        targets_test_df.append(target_test_df)

    # Check if all target_train and target_test have at least two classes
    all_have_two_classes = all(len(df.unique()) >= 2 for df in targets_train_df + targets_test_df)

    if all_have_two_classes:
        break


In [3502]:
trains_df[0].head(5)

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
0,1,1,57,0.0,0,160.0,55.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,NaN,1.0,7.0,110.0,60.0,65.0,14.0,37.8,4890.0,4210000.0,11.4,132000.0,99.0,29.0,0.94,338.0,31.0,33.0,1.08,NaN,136.0,3.4,8.40,1.15,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,1,0,0.0,0,0,0.0,NaN
1,0,0,62,0.0,0,159.0,70.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,NaN,1.0,7.0,140.0,90.0,124.0,14.0,38.5,5240.0,4980000.0,14.2,165000.0,104.0,40.0,1.00,260.0,31.0,39.0,0.82,NaN,140.0,3.9,9.40,1.19,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,1,0,0.0,0,0,0.0,NaN
2,1,0,72,0.0,0,175.0,83.0,1,0,0,0,0,0,1,0,0,0,0,0,0.0,0,0,1,1,0.0,7.0,0.2,2.0,80.0,60.0,87.0,NaN,39.0,6000.0,3010000.0,9.4,123000.0,107.0,38.0,0.91,474.0,18.0,18.0,1.75,0.36,139.0,4.0,8.13,1.10,369.0,7.41,30.4,2.6,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,1.0
3,1,0,53,0.0,0,177.0,96.0,1,1,0,0,0,0,0,0,0,0,1,0,0.0,0,1,2,1,0.0,572000.0,12.0,7.0,125.0,85.0,NaN,NaN,40.4,7040.0,4770000.0,14.4,19000.0,230.0,NaN,3.22,NaN,NaN,80.0,1.94,1.02,122.0,3.5,NaN,1.10,407.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,NaN
4,0,1,58,0.0,0,160.0,64.3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,1,1,1,0.0,602100.0,13.5,7.0,107.0,40.0,125.0,16.0,40.0,2810.0,4460.0,14.5,58000.0,115.0,48.0,1.30,NaN,NaN,37.0,1.48,NaN,135.0,3.9,NaN,1.10,455.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,0,0.0,0,0,0.0,NaN


In [3503]:
tests_df[0].head(5)

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
17,1,0,39,0.0,0,NaN,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0.0,1,0,2,1,0.0,NaN,0.8,1.0,127.0,80.0,92.0,20.0,39.8,6470.0,4700000.0,14.3,62000.0,139.0,25.0,0.80,196.0,28.0,40.0,3.70,0.6,135.0,3.9,9.3,1.00,NaN,NaN,NaN,NaN,1,0,0.0,0.0,0.0,1,0,0.0,0,0,0.0,3.0
24,0,0,77,0.0,0,NaN,NaN,1,0,0,0,1,0,0,0,0,0,0,0,0.0,0,0,1,1,0.0,NaN,0.2,8.0,120.0,70.0,72.0,NaN,36.0,4860.0,3770000.0,11.7,67000.0,97.0,24.0,0.90,NaN,78.0,62.0,1.89,0.1,136.0,4.3,8.7,1.09,316.0,NaN,NaN,NaN,0,0,1.0,1.0,0.0,0,0,0.0,0,1,0.0,NaN
27,1,1,50,0.0,0,170.0,70.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,NaN,2.0,3.0,110.0,66.0,112.0,24.0,38.7,7930.0,4550000.0,13.3,74000.0,106.0,NaN,0.82,NaN,42.0,33.0,1.58,NaN,134.0,3.6,9.3,1.05,NaN,NaN,NaN,NaN,0,1,0.0,1.0,0.0,0,0,0.0,0,0,0.0,NaN
41,1,1,64,0.0,0,190.0,103.0,1,1,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,14370.0,1.0,6.0,130.0,80.0,114.0,15.0,40.0,4890.0,4230000.0,12.6,84000.0,248.0,39.0,1.50,746.0,43.0,32.0,1.20,0.4,129.0,4.9,9.3,NaN,NaN,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,1,0.0,1,0,NaN,2.0
42,1,1,20,0.0,0,172.0,76.0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,1,1,0.0,6800.0,1.0,3.0,140.0,70.0,98.0,14.0,39.5,5000.0,500000.0,14.7,37000.0,NaN,19.0,0.90,576.0,29.0,36.0,2.20,0.8,138.0,3.7,NaN,1.25,527.0,NaN,NaN,NaN,1,0,1.0,1.0,1.0,0,1,0.0,1,0,NaN,NaN


**Filling method**

In [3504]:
feature_info = pd.read_csv("../data/feature_info.csv")

In [3505]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

# Define the data types for imputation
data_types = {
    'categorical': np.int64,  # Categorical data type
    'int': np.int64,          # Integer data type
    'float': np.float64       # Float data type
}

def data_fillingKNN(dataframe) : 


    ### Before everything, get rid of nan only columns 
    # Check if any column contains only empty values (NaNs)
    empty_columns = dataframe.columns[dataframe.isnull().all()]

    # Replace empty columns with zeros
    for column in empty_columns:
        dataframe[column] = 0
    ###

    # Identify the missing values in filtered_df
    missing_values = dataframe.isnull().sum()

    # Separate features based on their data types
    categorical_features = feature_info[feature_info['type'] == 'categorical']['feature'].tolist()
    int_features = feature_info[feature_info['type'] == 'int']['feature'].tolist()
    float_features = feature_info[feature_info['type'] == 'float']['feature'].tolist()

    # Impute missing values for each data type
    for data_type, features in [('categorical', categorical_features), ('int', int_features), ('float', float_features)]:
        # Filter features based on data type
        features_to_impute = [feature for feature in features if feature in missing_values.index and missing_values[feature] > 0]

        if len(features_to_impute) > 0:
            # Prepare the imputation array
            impute_array = dataframe[features_to_impute].values

            if data_type in ['categorical', 'int']:
                # Perform imputation for 'categorical' and 'int' features using median strategy
                imputer = KNNImputer(n_neighbors=7)#, weights='distance')
                imputed_values = imputer.fit_transform(impute_array)

                dataframe.loc[:, features_to_impute] = np.round(imputed_values).astype(data_types[data_type])
                
            elif data_type == 'float':
                # Perform imputation for 'float' features using mean strategy
                imputer = KNNImputer(n_neighbors=7)#, weights='distance')
                imputed_values = imputer.fit_transform(impute_array)

                dataframe.loc[:, features_to_impute] = imputed_values

    # Verify if any missing values remain in the DataFrame
    missing_values_after_imputation = dataframe.isnull().sum()
    missing_values_to_print = missing_values_after_imputation[missing_values_after_imputation != 0]
    if not missing_values_to_print.empty:
        print(f"There are still missing values in the DataFrame after imputation:\n{missing_values_to_print}")

    return dataframe

In [3506]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from scipy.sparse import csr_matrix

def STD_AND_knn_fill_num(df):

    empty_columns = df.columns[df.isna().all()]  # Find columns with all missing values

    for column in empty_columns:
        df[column] = 0  # Fill the entire column with zeros

    # Convert the DataFrame to a sparse matrix (csr_matrix)
    sparse_matrix = csr_matrix(df.values)

    # Convert the sparse matrix to a dense numpy array
    dense_array = sparse_matrix.toarray()

    # Initialize the KNN imputer
    knn_imputer = KNNImputer(n_neighbors=7)#, weights='distance')

    # Fit the imputer on the dense array
    knn_imputer.fit(dense_array)

    # Impute missing values
    imputed_array = knn_imputer.transform(dense_array)

    # Convert the imputed matrix back to a DataFrame
    imputed_df = pd.DataFrame(imputed_array, columns=df.columns, index=None)

    means = imputed_df.mean(axis=0)
    std = imputed_df.std(axis=0)

    imputed_df = (imputed_df - means) / std.where(std != 0, 1)
    

    return imputed_df

In [3507]:
# count_of_190 = (trains_df[0]['Altezza'] == 190).sum()
# print("Count of values equal to 190 in train_df[0]['Altezza']: ", count_of_190)

# # Assuming train_df is your DataFrame
# missing_values_count = trains_df[0]['Altezza'].isna().sum()
# print("Count of missing values in train_df[0]['Altezza']: ", missing_values_count)

In [3508]:
# Apply data_fillingKNN to each dataframe in trains_df and tests_df
for i in range(len(trains_df)):
    trains_df[i] = data_fillingKNN(trains_df[i])


for i in range(len(tests_df)):
    tests_df[i] = data_fillingKNN(tests_df[i])

In [3509]:
# count_of_190 = (trains_df[0]['Altezza'] == 190).sum()
# print("Count of values equal to 190 in train_df[0]['Altezza']: ", count_of_190)

# # Assuming train_df is your DataFrame
# missing_values_count = trains_df[0]['Altezza'].isna().sum()
# print("Count of missing values in train_df[0]['Altezza']: ", missing_values_count)

**Standardize**

In [3510]:
def standardize(df) : 
    means = df.mean(axis=0)
    std = df.std(axis=0)

    std_df = (df - means) / std.where(std != 0, 1)

    return std_df

# Apply standardize to each dataframe in trains_df and tests_df
for i in range(len(trains_df)):
    trains_df[i] = standardize(trains_df[i])

for i in range(len(tests_df)):
    tests_df[i] = standardize(tests_df[i])

In [3511]:
# from sklearn.preprocessing import StandardScaler

# scalers = [StandardScaler() for _ in range(num_folds)]

# for i in range(len(trains_df)):
    
#     scaled_data = scalers[i].fit_transform(trains_df[i])
#     trains_df[i] = pd.DataFrame(scaled_data, columns=trains_df[i].columns, index=trains_df[i].index)

#     # Scale test because not already done 
#     scaled_test_data = scalers[i].transform(tests_df[i])
#     tests_df[i] = pd.DataFrame(scaled_test_data, columns=tests_df[i].columns, index=tests_df[i].index)


In [3512]:
trains_df[0].head(10)

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
0,0.727659,0.896965,0.467922,-0.233626,-0.184141,-1.124134,-1.370410,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,-0.612450,0.657670,-0.075100,-0.598533,-1.784976,-0.772558,-0.352229,-0.193339,0.298730,-0.308557,1.104780,-0.275114,-0.506895,-0.487156,-0.530378,-0.720150,-0.558564,-0.481745,NaN,0.085103,-0.663093,-0.107211,-0.184938,NaN,NaN,NaN,NaN,-1.717556,-0.184141,-1.361089,-1.485507,-1.05021,3.288870,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
1,-1.351366,-1.096290,0.800568,-0.233626,-0.184141,-1.230073,-0.452634,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,-0.612450,0.657670,1.356223,1.569273,1.350765,-0.772558,0.186236,-0.057840,0.810369,0.973882,1.709227,-0.160431,-0.332051,-0.439024,-0.653288,-0.720150,-0.419460,-0.534970,NaN,0.976653,0.296647,1.229289,0.045967,NaN,NaN,NaN,NaN,-1.717556,-0.184141,-1.361089,-1.485507,-1.05021,3.288870,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
2,0.727659,-1.096290,1.465859,-0.233626,-0.184141,0.464956,0.342772,1.025256,-0.388950,-0.265025,-0.227496,-0.227496,-1.717556,2.217356,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,-0.616081,-0.757235,-0.880916,-1.506424,-0.598533,-0.615716,NaN,0.570853,0.236386,-0.498629,-1.224584,0.939930,-0.091621,-0.363841,-0.511222,-0.316075,-0.994605,-0.906325,-0.344587,-0.574507,0.753765,0.488595,-0.468065,-0.473570,-0.529034,-0.603384,1.394303,0.35703,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,-0.850265
3,0.727659,-1.096290,0.201805,-0.233626,-0.184141,0.676835,1.138178,1.025256,2.528175,-0.265025,-0.227496,-0.227496,-1.717556,-0.443471,-0.184141,-0.265025,-0.416569,2.528175,-0.227496,-0.267385,-0.521523,1.983263,0.099979,0.129099,-0.131306,1.622824,1.378341,0.657670,0.640561,1.207972,NaN,NaN,1.647782,0.639012,0.670831,1.065485,-0.964995,2.729575,NaN,1.341868,NaN,NaN,0.531087,-0.305692,-0.349774,-3.035323,-0.471145,NaN,-0.473570,-0.299603,NaN,NaN,NaN,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
4,-1.351366,0.896965,0.534451,-0.233626,-0.184141,-1.124134,-0.801389,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,-1.717556,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,1.983263,-0.649862,0.129099,-0.131306,1.740642,1.649812,0.657670,-0.218233,-2.043737,1.403913,-0.240633,1.340088,-0.998591,-2.495708,1.111286,-0.250648,0.091871,-0.204892,-0.198363,NaN,NaN,-0.465828,-0.399860,NaN,-0.137785,0.296647,NaN,-0.473570,-0.009794,NaN,NaN,NaN,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
5,0.727659,-1.096290,0.667509,-0.233626,-0.184141,-0.170680,-0.146708,1.025256,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,2.21

In [3513]:
targets_train_df[0].head(10)

0    0
1    0
2    0
3    0
4    0
5    1
6    0
7    0
8    0
9    0
Name: ICU, dtype: int64

**Oversampling method**

In [3514]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

In [3515]:
def RandomMinorityOversample(df, associated_target): 
    # Check if there's only one class in the target variable
    unique_classes = set(associated_target)
    if len(unique_classes) == 1:
        # Return the original data without oversampling
        return df, associated_target

    # Oversample the minority class in the training data
    oversampler = RandomOverSampler(sampling_strategy='not majority', random_state=42)
    df_resampled, associated_target_resampled = oversampler.fit_resample(df, associated_target)
    return df_resampled, associated_target_resampled

def smote_oversampling(df, associated_target):
    X = df
    y = associated_target

    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    associated_target_ressampled = pd.Series(y_resampled, name="ICU")

    return df_resampled, associated_target_ressampled

def adasyn_oversampling(df, associated_target):
    X = df
    y = associated_target

    adasyn = ADASYN(random_state=42)
    X_resampled, y_resampled = adasyn.fit_resample(X, y)

    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    associated_target_ressampled = pd.Series(y_resampled, name="ICU")

    return df_resampled, associated_target_ressampled

# Apply oversampling to each dataframe and target
for i in range(len(trains_df)):
    trains_df[i], targets_train_df[i] = smote_oversampling(trains_df[i], targets_train_df[i])

# for i in range(len(tests_df)):
#     tests_df[i], targets_test_df[i] = oversample(tests_df[i], targets_test_df[i])



In [3516]:
trains_df[0].head(10)

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
0,0.727659,0.896965,0.467922,-0.233626,-0.184141,-1.124134,-1.370410,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,-0.612450,0.657670,-0.075100,-0.598533,-1.784976,-0.772558,-0.352229,-0.193339,0.298730,-0.308557,1.104780,-0.275114,-0.506895,-0.487156,-0.530378,-0.720150,-0.558564,-0.481745,NaN,0.085103,-0.663093,-0.107211,-0.184938,NaN,NaN,NaN,NaN,-1.717556,-0.184141,-1.361089,-1.485507,-1.05021,3.288870,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
1,-1.351366,-1.096290,0.800568,-0.233626,-0.184141,-1.230073,-0.452634,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,-0.612450,0.657670,1.356223,1.569273,1.350765,-0.772558,0.186236,-0.057840,0.810369,0.973882,1.709227,-0.160431,-0.332051,-0.439024,-0.653288,-0.720150,-0.419460,-0.534970,NaN,0.976653,0.296647,1.229289,0.045967,NaN,NaN,NaN,NaN,-1.717556,-0.184141,-1.361089,-1.485507,-1.05021,3.288870,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
2,0.727659,-1.096290,1.465859,-0.233626,-0.184141,0.464956,0.342772,1.025256,-0.388950,-0.265025,-0.227496,-0.227496,-1.717556,2.217356,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,-0.616081,-0.757235,-0.880916,-1.506424,-0.598533,-0.615716,NaN,0.570853,0.236386,-0.498629,-1.224584,0.939930,-0.091621,-0.363841,-0.511222,-0.316075,-0.994605,-0.906325,-0.344587,-0.574507,0.753765,0.488595,-0.468065,-0.473570,-0.529034,-0.603384,1.394303,0.35703,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,-0.850265
3,0.727659,-1.096290,0.201805,-0.233626,-0.184141,0.676835,1.138178,1.025256,2.528175,-0.265025,-0.227496,-0.227496,-1.717556,-0.443471,-0.184141,-0.265025,-0.416569,2.528175,-0.227496,-0.267385,-0.521523,1.983263,0.099979,0.129099,-0.131306,1.622824,1.378341,0.657670,0.640561,1.207972,NaN,NaN,1.647782,0.639012,0.670831,1.065485,-0.964995,2.729575,NaN,1.341868,NaN,NaN,0.531087,-0.305692,-0.349774,-3.035323,-0.471145,NaN,-0.473570,-0.299603,NaN,NaN,NaN,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
4,-1.351366,0.896965,0.534451,-0.233626,-0.184141,-1.124134,-0.801389,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,-1.717556,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,1.983263,-0.649862,0.129099,-0.131306,1.740642,1.649812,0.657670,-0.218233,-2.043737,1.403913,-0.240633,1.340088,-0.998591,-2.495708,1.111286,-0.250648,0.091871,-0.204892,-0.198363,NaN,NaN,-0.465828,-0.399860,NaN,-0.137785,0.296647,NaN,-0.473570,-0.009794,NaN,NaN,NaN,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
5,0.727659,-1.096290,0.667509,-0.233626,-0.184141,-0.170680,-0.146708,1.025256,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,2.21

In [3517]:
targets_train_df[0].head(10)

0    0
1    0
2    0
3    0
4    0
5    1
6    0
7    0
8    0
9    0
Name: ICU, dtype: int64

In [3518]:
for i in range(num_folds):
    trains_df[i] = trains_df[i].reset_index(drop=True)
    tests_df[i] = tests_df[i].reset_index(drop=True)
    targets_train_df[i] = targets_train_df[i].reset_index(drop=True)
    targets_test_df[i] = targets_test_df[i].reset_index(drop=True)
    

**Data augmentation**

In [3519]:
import pandas as pd
import numpy as np

def mixup_augmentation(dataframe, labels, alpha=0.5):
    """
    Effectue l'augmentation de données par mixup sur une DataFrame en utilisant les étiquettes pour le mélange.

    Args:
        dataframe (pandas.DataFrame): La DataFrame contenant les caractéristiques (features) à augmenter.
        labels (pandas.Series): Le vecteur d'étiquettes correspondant aux exemples dans la DataFrame.
        

    Returns:
        pandas.DataFrame: Une nouvelle DataFrame augmentée par mixup.
        pandas.Series: Le vecteur d'étiquettes correspondant aux exemples augmentés.
    """
    # Créer une copie de la DataFrame originale pour stocker les données augmentées
    augmented_df = dataframe.copy()

    # Créer un dictionnaire pour regrouper les indices par étiquette
    label_indices = {}
    for idx, label in enumerate(labels):
        if label not in label_indices:
            label_indices[label] = []
        label_indices[label].append(idx)

    # Appliquer le mixup pour chaque paire d'exemples ayant la même étiquette
    for label, indices in label_indices.items():
        for idx in indices:
            # Sélectionner un exemple aléatoire avec la même étiquette
            random_index = np.random.choice(indices)

            # Ignorer si c'est le même exemple
            if idx == random_index:
                continue

            

            # Mélanger les caractéristiques
            mixed_features = dataframe.iloc[idx, :] * alpha + (1 - alpha) * dataframe.iloc[random_index, :]

            # Ajouter l'exemple mixé à la DataFrame augmentée
            augmented_df = augmented_df.append(mixed_features, ignore_index=True)

            # Ajouter l'étiquette correspondante à l'exemple mixé, but only if idx != random_index
            if idx != random_index:
                if labels[idx] != labels[random_index] :
                    raise
                new_labels = labels[idx] 
                labels = labels.append(pd.Series(new_labels), ignore_index=True)

    if len(labels) != augmented_df.shape[0]:
        raise ValueError("The length of augmented labels does not match the shape of augmented DataFrame.")

    return augmented_df, labels


In [3520]:
trains_df[0].head(5)

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
0,0.727659,0.896965,0.467922,-0.233626,-0.184141,-1.124134,-1.370410,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,-0.612450,0.657670,-0.075100,-0.598533,-1.784976,-0.772558,-0.352229,-0.193339,0.298730,-0.308557,1.104780,-0.275114,-0.506895,-0.487156,-0.530378,-0.720150,-0.558564,-0.481745,NaN,0.085103,-0.663093,-0.107211,-0.184938,NaN,NaN,NaN,NaN,-1.717556,-0.184141,-1.361089,-1.485507,-1.05021,3.288870,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
1,-1.351366,-1.096290,0.800568,-0.233626,-0.184141,-1.230073,-0.452634,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,-0.612450,0.657670,1.356223,1.569273,1.350765,-0.772558,0.186236,-0.057840,0.810369,0.973882,1.709227,-0.160431,-0.332051,-0.439024,-0.653288,-0.720150,-0.419460,-0.534970,NaN,0.976653,0.296647,1.229289,0.045967,NaN,NaN,NaN,NaN,-1.717556,-0.184141,-1.361089,-1.485507,-1.05021,3.288870,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
2,0.727659,-1.096290,1.465859,-0.233626,-0.184141,0.464956,0.342772,1.025256,-0.388950,-0.265025,-0.227496,-0.227496,-1.717556,2.217356,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,-0.616081,-0.757235,-0.880916,-1.506424,-0.598533,-0.615716,NaN,0.570853,0.236386,-0.498629,-1.224584,0.939930,-0.091621,-0.363841,-0.511222,-0.316075,-0.994605,-0.906325,-0.344587,-0.574507,0.753765,0.488595,-0.468065,-0.473570,-0.529034,-0.603384,1.394303,0.35703,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,-0.850265
3,0.727659,-1.096290,0.201805,-0.233626,-0.184141,0.676835,1.138178,1.025256,2.528175,-0.265025,-0.227496,-0.227496,-1.717556,-0.443471,-0.184141,-0.265025,-0.416569,2.528175,-0.227496,-0.267385,-0.521523,1.983263,0.099979,0.129099,-0.131306,1.622824,1.378341,0.657670,0.640561,1.207972,NaN,NaN,1.647782,0.639012,0.670831,1.065485,-0.964995,2.729575,NaN,1.341868,NaN,NaN,0.531087,-0.305692,-0.349774,-3.035323,-0.471145,NaN,-0.473570,-0.299603,NaN,NaN,NaN,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN
4,-1.351366,0.896965,0.534451,-0.233626,-0.184141,-1.124134,-0.801389,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,-1.717556,-0.443471,-0.184141,-0.265025,-0.416569,-0.388950,-0.227496,-0.267385,-0.521523,1.983263,-0.649862,0.129099,-0.131306,1.740642,1.649812,0.657670,-0.218233,-2.043737,1.403913,-0.240633,1.340088,-0.998591,-2.495708,1.111286,-0.250648,0.091871,-0.204892,-0.198363,NaN,NaN,-0.465828,-0.399860,NaN,-0.137785,0.296647,NaN,-0.473570,-0.009794,NaN,NaN,NaN,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,-0.171499,NaN


In [3521]:
def doubleWithNoise(df, target) :


    # Determine the number of samples to add (doubling the size)
    num_samples_to_add = 5 * len(df)
    num_columns = len(df.columns)

    # Generate random noise with mean 0 and variance 1
    noise = 0.6 * np.random.normal(0, 1, (num_samples_to_add, num_columns))
    # print("noise = "+str(noise))

    # Randomly select samples from the original DataFrame
    random_indices = np.random.choice(len(df), num_samples_to_add, replace=True)
    selected_samples = df.iloc[random_indices]
    selected_targets = target.iloc[random_indices]

    # Add noise to the selected samples
    new_samples = selected_samples + noise

    # Concatenate the original DataFrame with the new samples
    doubled_df = pd.concat([df, new_samples])
    doubled_target = pd.concat([target, selected_targets])
    

    return doubled_df, doubled_target


# for i in range(num_folds):
#     trains_df[i], targets_train_df[i] = doubleWithNoise(trains_df[i], targets_train_df[i])

In [3522]:
trains_df[0].shape[0]

60

In [3523]:
for i in range(num_folds):
    trains_df[i], targets_train_df[i] = doubleWithNoise(trains_df[i], targets_train_df[i])

# for i in range(num_folds):
#     trains_df[i], targets_train_df[i] = mixup_augmentation(trains_df[i], targets_train_df[i])

In [3524]:
trains_df[0].shape[0]

60

In [3525]:
trains_df[0].tail(5)

,Sesso,Nazionalità,Età,Domicilio,Profilassi,Altezza,Peso,Comorbilità,Diabete,HIV,IRC,GCS,Prostrazione,Shock,Bleeding,ARDS,Anemia,Creatinina,Glicemia,Acidosi,Bilirubina,Hyperparasitaemia,Numerocriteri,Falciparum,Ovale,Parassitemiavaloreassoluto,Percentualeparassitemia,Ritardoterapeutico,PAsistolica,PAdiastolica,FC,FR,Temperatura,T0GBincellul,T0GRincellul,T0Hbingdl,T0PLTincellul,T0Glicemiainmgdl,T0Azotemiainmgdl,T0Creatininainmgdl,T0LDHinUL,T0ASTinUL,T0ALTinUL,T0Bilirubinatotinmgdl,T0Bilirubinadirettainmgdl,T0SodioinmEql,T0PotassioinmEql,T0Cainmgdl,T0INR,T0fibrinogenoinmgdl,T0pH,T0bicarbonatiinmmoll,T0Lattatiinmmoll,ArtesunatoevSomministrazione1Datainizioeora,ArtemetherLumefantrinaSomministrazione1Datainizioeora,DiidroartemisininaPiperachinaSomministrazione1Datainizioeora,ACT,ArtesunatoACT,AtovaquoneProguanile,Doxiciclinaperos,Clindamicina,Antibiotici,Chinino,Primachina,DurataArtesunato
55,-1.351366,0.896965,0.334863,4.205260,-0.184141,NaN,NaN,1.025256,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.38895,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,0.781103,-0.573199,-1.315581,-1.104354,-1.466087,-0.772558,-1.736852,-0.259153,0.032944,-1.178783,-0.323914,-0.252177,-0.697634,-0.647597,-0.385408,NaN,-0.952693,-0.518593,NaN,1.199540,0.488595,-0.107211,-0.184938,-0.522996,NaN,NaN,NaN,0.572519,-0.184141,-1.361089,-1.485507,-1.05021,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,NaN,NaN
56,0.727659,-1.096290,0.135276,-0.233626,-0.184141,NaN,NaN,1.025256,2.528175,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.38895,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,NaN,2.503974,-0.313654,-0.237232,0.394099,-0.506595,-0.582999,-1.513487,-0.691324,-1.682598,-0.415497,-0.068685,-0.157207,-0.198363,0.394594,NaN,-0.697668,5.070062,NaN,-1.697998,-0.279197,-0.775460,4.721805,0.207562,1.070616,-0.432526,-0.171053,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,NaN,NaN
57,-1.351366,0.896965,-0.796132,-0.233626,-0.184141,NaN,1.382919,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.38895,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,NaN,-0.359077,0.657670,-1.506424,-0.598533,0.128358,NaN,-0.198382,-0.770178,0.179126,-0.262755,-0.580347,0.023062,-0.619749,-0.679685,-0.306620,-0.551255,-0.929509,-0.272938,-0.458736,0.307990,-0.663093,NaN,-0.473570,NaN,NaN,NaN,NaN,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,1.402379,-0.298988,NaN,NaN
58,0.727659,-1.096290,1.399330,-0.233626,-0.184141,NaN,-0.513819,1.025256,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.38895,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,0.008381,-0.250488,-0.573199,1.356223,0.846671,-0.031087,NaN,-0.198382,-0.150754,1.082800,1.935711,0.665181,-0.183367,0.055785,-0.439024,-0.364923,0.398780,0.391982,-0.457179,-0.560887,-0.583560,-0.663093,NaN,-1.050834,0.014356,NaN,NaN,NaN,0.572519,-0.184141,0.720577,0.660225,0.93352,-0.298988,-0.572519,-0.301701,-0.701189,-0.298988,NaN,NaN
59,-1.351366,0.896965,-1.594482,-0.233626,-0.184141,NaN,0.281587,-0.959110,-0.388950,-0.265025,-0.227496,-0.227496,0.572519,-0.443471,-0.184141,-0.265025,-0.416569,-0.38895,-0.227496,-0.267385,-0.521523,-0.495816,-0.649862,0.129099,-0.131306,-0.151100,-0.178096,-1.188633,-0.075100,-0.598533,0.234654,NaN,-1.736852,0.197672,-0.159751,-0.858173,0.775081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.572519,-0.184141,NaN,NaN,NaN,-0.298988,-0.572519,3.258366,-0.701189,-0.298988,NaN,NaN


**PCA**

In [3526]:
from sklearn.decomposition import PCA

def PrinCompAnalysis(df_train, df_test):
    # Créer et entraîner le modèle PCA sur les données d'entraînement
    n_components = 5  # Remplacez le nombre de composantes principales souhaité
    pca = PCA(n_components=n_components)
    pca.fit(df_train)

    # Appliquer la transformation PCA sur les données d'entraînement et de test
    X_train_pca = pca.transform(df_train)
    X_test_pca = pca.transform(df_test)

    # Créer des DataFrames pour les données transformées
    X_train_pca_df = pd.DataFrame(X_train_pca, columns=[f'PC{i}' for i in range(1, n_components + 1)])
    X_test_pca_df = pd.DataFrame(X_test_pca, columns=[f'PC{i}' for i in range(1, n_components + 1)])

    return X_train_pca_df, X_test_pca_df


# # Apply PCA to each train & test df
# for i in range(num_folds):    
    
#     trains_df[i], tests_df[i] = PrinCompAnalysis(trains_df[i], tests_df[i])



**Evaluation**

In [3527]:
from plot_risk_index import corr_risk_uncer

sum_corr = 0

for i in range(num_folds) : 
    corr = corr_risk_uncer(trains_df[i], targets_train_df[i], tests_df[i], targets_test_df[i])
    print("corr = "+str(corr))
    sum_corr += corr

avg_corr = sum_corr / num_folds

print("avg corr = "+str(avg_corr))

corr = 0.7569165755759636
corr = 0.4261191642069159
corr = 0.8223453249651986
corr = 0.4748845219025156
corr = 0.8432908421628724
corr = 0.13354299821309848
avg corr = 0.5761832378377608


In [3528]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

def evaluate(train_df_resampled, target_train_df_resampled, test_df_resampled, target_test_df_resampled) :
    # Initialize classifiers
    classifiers = {
        'Decision Tree (C4.5)': DecisionTreeClassifier(),
        'k-Nearest Neighbor (kNN)': KNeighborsClassifier(),
        'Logistic Regression (LR)': LogisticRegression(),
        'Naïve Bayes (NB)': GaussianNB(),
        'Random Forest (RF)': RandomForestClassifier(),
        'Support Vector Machine (SVM)': SVC(probability=True)
    }

    # Initialize lists to store AUC scores
    auc_scores = []

    # Iterate through the classifiers
    for classifier_name, classifier in classifiers.items():
        # print(f"Training and testing {classifier_name}...")
        
        # Split data into train and test sets
        X_train = train_df_resampled
        y_train = target_train_df_resampled
        X_test = test_df_resampled
        y_test = target_test_df_resampled

        # Train the classifier on the training data
        classifier.fit(X_train, y_train)

        # Predict probabilities for the positive class
        y_prob = classifier.predict_proba(X_test)[:, 1]

        # Calculate AUC score for this fold
        auc = roc_auc_score(y_test, y_prob)
        
        # Append average AUC score to the overall list
        auc_scores.append(auc)

    return np.mean(auc_scores)


In [3529]:
# Apply oversampling to each dataframe and target

sum_avg_auc = 0

for i in range(num_folds):
    auc = evaluate(trains_df[i], targets_train_df[i], tests_df[i], targets_test_df[i])
    print("auc for fold set "+str(i+1)+" : "+str(auc))
    sum_avg_auc += auc

print("avg avg auc = "+str(sum_avg_auc / num_folds))

print("avg corr risk index = "+str(avg_corr))

ValueError: Input X contains NaN.
DecisionTreeClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values